In [9]:
import numpy as np
import pandas as pd
import time
import math
import sys
import random
from itertools import combinations
from helper_functions_assignment2 import*

def ALNS(min_weight, reaction_factor, iterations_per_phase, number_of_phases,
        node_impact, node_degree, edge_assignment, edge_weights, plex_assignment, s):

    start_time = time.time()
    end_time = start_time + (60*4) # 4 Minutes

    destroy_methods = [remove_random_node, remove_highest_cost_node, remove_most_edges_node, 
                   remove_smallest_splex, remove_largest_splex]
    repair_methods = [add_to_largest_splex, add_to_smallest_splex, add_new_splex, add_to_random_splex]

    destroy_weights = np.ones(len(destroy_methods))
    repair_weights = np.ones(len(repair_methods))

    n = len(plex_assignment)

    current_score = sum(node_impact)/2
    
    # initialize score trajectory
    traj = []
    traj.append(current_score)

    total_destroy_executions = np.zeros(len(destroy_methods))
    total_repair_executions = np.zeros(len(repair_methods))
    total_destroy_successes = np.zeros(len(destroy_methods))
    total_repair_successes = np.zeros(len(repair_methods))

    for i in range(number_of_phases):
        print(i)

        destroy_prob = destroy_weights/destroy_weights.sum()
        repair_prob = repair_weights/repair_weights.sum()

        #print(destroy_prob)
        #print(repair_prob)

        destroy_applications = np.zeros(len(destroy_methods))
        destroy_successes = np.zeros(len(destroy_methods))
        repair_applications = np.zeros(len(repair_methods))
        repair_successes = np.zeros(len(repair_methods))

        for j in range(iterations_per_phase):
            destroy_idx = np.random.choice(len(destroy_methods), p=destroy_prob)
            repair_idx = np.random.choice(len(repair_methods), p=repair_prob)
            
            destroy_applications[destroy_idx] += 1
            total_destroy_executions[destroy_idx] += 1
            repair_applications[repair_idx] += 1
            total_repair_executions[repair_idx] += 1

            plex_assignment_new = plex_assignment.copy()
            edge_assignment_new = edge_assignment.copy()
            node_impact_new = node_impact.copy()
            node_degree_new = node_degree.copy()

            _, _, removed_nodes = destroy_methods[destroy_idx](plex_assignment_new, edge_assignment_new, n, node_impact_new, node_degree_new, edge_weights)
            
            _, _ = repair_methods[repair_idx](plex_assignment_new, edge_assignment_new, n, removed_nodes, node_impact_new, node_degree_new, edge_weights, s)

            new_score = sum(node_impact_new)/2

            if new_score < current_score:
                destroy_successes[destroy_idx] += 1
                repair_successes[repair_idx] += 1
                total_destroy_successes[destroy_idx] += 1
                total_repair_successes[repair_idx] += 1
            
                current_score = new_score
                plex_assignment = plex_assignment_new
                edge_assignment = edge_assignment_new
                node_impact = node_impact_new
                node_degree = node_degree_new
            if time.time() > end_time:
                break
            #print(1)
            # append current score
            traj.append(current_score)
            
        if time.time() > end_time:

            break

        for j in range(len(destroy_methods)):
            destroy_weights[j] = update_weight(destroy_weights[j], destroy_applications[j], destroy_successes[j], reaction_factor, min_weight)
            #print("-" + str(j) + ": " + str(destroy_weights[j]))
            
        for j in range(len(repair_methods)):
            repair_weights[j] = update_weight(repair_weights[j], repair_applications[j], repair_successes[j], reaction_factor, min_weight)
            #print("-" + str(j) + ": " + str(repair_weights[j]))
    
    print("destroy: " + str(total_destroy_successes) + "/" + str(total_destroy_executions))
    print("repair:" + str(total_repair_successes) + "/" + str(total_repair_executions))
    print("score: " + str(current_score))

In [10]:
instances = ["heur049_n_300_m_17695", "heur050_n_300_m_19207", "heur051_n_300_m_20122"]

min_weight = 0.067264
reaction_factor = 0.036799
iterations_per_phase = 89

for instance in instances:
    path = "data/inst_competition/" + instance +".txt"
    node_impact, node_degree, plex_assignment, edges_n1, edges_n2, edge_weights, edge_assignment, s, n, m = create_problem_instance(path)
    plex_assignment = np.random.choice(10, n)
    repair_solution(node_impact, node_degree, plex_assignment, edge_weights, edge_assignment, s)
    number_of_phases = round(10000 / iterations_per_phase)
    
    ALNS(min_weight, reaction_factor, iterations_per_phase, number_of_phases, node_impact, node_degree, edge_assignment, edge_weights, plex_assignment, s)
    
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
destroy: [ 77. 241.  24.  76.  45.]/[642. 785. 529. 649. 574.]
repair:[ 57. 144. 122. 140.]/[715. 827. 814. 823.]
score: 32292.0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
destroy: [ 61. 197.   3.  38.  11.]/[319. 334. 310. 293. 264.]
repair:[30. 98. 91. 91.]/[331. 432. 399. 358.]
score: 39921.0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
destroy: [ 56. 174.  15.  61.  33.]/[252. 310. 275. 233. 254.]
repair:[ 26.  96.  97. 120.]/[327. 303. 344. 350.]
score: 25172.0
